In [1]:
from pprint import pprint

import numpy as np
import pandas as pd
import shap
import torch
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from torch import nn
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    OPTForSequenceClassification,
    Pipeline,
)

import wandb

MODEL = "facebook/opt-350m"
MAX_POSITION_EMBEDDINGS = 2048

from dataclasses import dataclass

/h/u11/c0/00/ammcourt/miniconda3/envs/csc413/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

print(device)

cuda


In [3]:
CHECKPOINT_DIR = "OPT-350m-mimic-full"
VAL_DATASET_PATH = "data/val_9.csv"
CODE_PATH = "data/icd9_codes.csv"

In [4]:
# Load dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True, device=device)

code_labels = pd.read_csv("data/icd9_codes.csv")
dataset = load_dataset("csv", data_files=VAL_DATASET_PATH)

# Create class dictionaries
classes = [class_ for class_ in code_labels["icd_code"] if class_]
class2id = {class_: id for id, class_ in enumerate(classes)}
id2class = {id: class_ for class_, id in class2id.items()}


def multi_labels_to_ids(labels: list[str]) -> list[float]:
    ids = [0.0] * len(class2id)  # BCELoss requires float as target type
    for label in labels:
        ids[class2id[label]] = 1.0
    return ids


def preprocess_function(example):
    result = tokenizer(
        example["text"], truncation=True, max_length=MAX_POSITION_EMBEDDINGS
    )
    result["labels"] = [multi_labels_to_ids(eval(label)) for label in example["labels"]]
    return result


dataset = dataset.map(
    preprocess_function, load_from_cache_file=True, batched=True, num_proc=8
)

In [5]:
config, unused_kwargs = AutoConfig.from_pretrained(
    MODEL,
    num_labels=len(classes),
    id2label=id2class,
    label2id=class2id,
    problem_type="multi_label_classification",
    return_unused_kwargs=True,
)

if unused_kwargs:
    print(f"Unused kwargs: {unused_kwargs}")

model = OPTForSequenceClassification.from_pretrained(
    MODEL,
    config=config,
).to(device)

model.load_adapter(CHECKPOINT_DIR)
model.to_bettertransformer()

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttentionLayerBetterTransformer(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_fea

In [6]:
untokenized_dataset = load_dataset("csv", data_files=VAL_DATASET_PATH)

print(untokenized_dataset["train"][0])

{'text': "Sex:   M\n \nService: SURGERY\n \nAllergies: \nGrass ___, Standard / Lactose\n \n ___.\n \nChief Complaint:\nright popliteal aneurysm\n \nMajor Surgical or Invasive Procedure:\n___: popliteal artery stent graft\n\n \nHistory of Present Illness:\nMr. ___ has a fairly focal aneurysm in the\nmid right popliteal artery and is mostly full of thrombus and\nmeasures 3.1 cm.  It is patent and does have palpable pedal\npulse distally.  He has need of upcoming ankle surgery as well.\nHe has a past medical history notable for breast cancer status\npost mastectomy and chemotherapy/radiation therapy with duodenal\nulcer, pseudogout, depression, hypothyroidism, microvascular\ncerebrovascular disease, hyperlipidemia, and COPD.  He had vein\nmapping performed today which shows the lesser saphenous veins\nto be small and noncompressible bilaterally.  He has the\nthrombosis of the right greater saphenous at the level of the\nknee.  The left greater saphenous is adequate as are both\nbasilic an

In [7]:
inputs = tokenizer(
    untokenized_dataset["train"][0]["text"],
    return_tensors="pt",
    truncation=True,
    max_length=MAX_POSITION_EMBEDDINGS,
).to(device)

with torch.no_grad():
    logits = model(**inputs).logits

In [8]:
class OPT_ICD9_Pipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        return self.tokenizer(
            text,
            truncation=True,
            max_length=MAX_POSITION_EMBEDDINGS,
            return_tensors="pt",
        )

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        return outputs

    def postprocess(self, model_outputs):
        # logits = model_outputs.logits[0].numpy()
        #print(logits)
        probs = model_outputs["logits"].sigmoid()

        output = []
        for i, prob in enumerate(probs[0]):
            label = self.model.config.id2label[i]
            score = prob
            output.append({"label": label, "score": score})
        return output

In [9]:
pipeline = OPT_ICD9_Pipeline(model=model, tokenizer=tokenizer, device=device)

In [10]:
model.device

device(type='cuda', index=0)

In [ ]:
pipeline(untokenized_dataset["train"][2]["text"])

In [12]:
masker = shap.maskers.Text(pipeline.tokenizer)

In [13]:
sample = shap.sample(untokenized_dataset["train"]["text"], 2)

In [14]:
explainer = shap.Explainer(pipeline, masker)

In [ ]:
untokenized_dataset["train"][:2]["text"]

In [ ]:
shap_values = explainer(untokenized_dataset["train"][:5]["text"])

In [ ]:
shap.plots.text(shap_values[0, :, "d-2749"])

In [ ]:
out = shap.plots.text(shap_values[0, :, "d-2749"], display = False)
print(out)

In [ ]:
# print(shap_values)

print(explainer.feature_names)
print(len(shap_values.values[1]))
# print(shap_values[0, :, "d-2749"].values)
print(len(shap_values[0, :, "d-2749"].values))
# print(len(shap_values.values[0]))
# print(len(shap_values.base_values[0]))
# print(len(shap_values.data[0]))

# print(shap_values.values[0])
# print(shap_values.base_values[0])
# print(shap_values.data[0])

max_indices = []
for val in shap_values.values[0]:
    print(val)
    inde = np.argmax(val)
    if inde not in max_indices:
        max_indices.append(inde)
    
print(max_indices)

# feature_names = untokenized_dataset["train"].columns
# rf_resultX = pd.DataFrame(shap_values, columns = feature_names)

# vals = np.abs(rf_resultX.values).mean(0)

# shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
#                                   columns=['col_name','feature_importance_vals'])
# shap_importance.sort_values(by=['feature_importance_vals'],
#                                ascending=False, inplace=True)
# shap_importance.head()


# values, clustering = unpack_shap_explanation_contents(v)
#             tokens, values, group_sizes = process_shap_values(v.data, values, grouping_threshold, separator, clustering)


Shap pipeline works in the following order:
- Generate the indices array (note this array corresponds to tokens, not strings)
- Tokenize the input dataset
- Pass the tokenized input dataset to the masking functions along with the indices
- The pass all tokenized and masked tokenized datasets to the faithfulness calcuation

This requires a prediction function that expects a tokenized input

In [27]:
def get_index_arrays_shap(inputs, pred_func, model, tokenizer, top_k = 5):
    """ Function to create the arrays corresponding to the shap 
    
    The output is in the format [[input_index_0, input_index_0, ... input_index_n, input_index_n], 
    [rationale_token_index_0 (for input 0), rationale_token_index_1 (for input 0), ... rationale_token_index_k-1 (for input n), rationale_token_index_k (for input n)]]. 
    This is used as an indexing array for masking.
    
    """
    
    # get the shap values over the inputs
    # shap_values = explainer(inputs, batch_size=5)
    
    # get the mode inferences for the inputs
    inferences = pred_func(inputs, model, tokenizer)
    indices_array = None
    # get the longest 
    
    for i, val in enumerate(shap_values):
        # get the choosen labels
        print("Inferences: ", inferences)
        choosen_labels = np.where(inferences[i] > 0.5)
        choosen_labels = np.unique(choosen_labels)
        
        # convert the indices to labels
        choosen_labels = np.array(choosen_labels).astype(int)
        print("Choosen label: ", choosen_labels)
        choosen_labels = [id2class[label] for label in choosen_labels]
        print(id2class)
        print(choosen_labels)
        
        # for each shap value, index in via it's choosen labels
        total_top_k_indices = np.array([])
        top_val = min(top_k, len(choosen_labels))
        print(top_val)
        print(choosen_labels)
        
        
        for label in choosen_labels:
            # get the top k shap value indices
            print(label)
            top_k_indices = np.argsort(shap_values[i, :, label].values)[-top_val:]
            print(top_k_indices)
            total_top_k_indices = np.append(total_top_k_indices, top_k_indices)
        
        # sort the indices array to be in ascending order
        total_top_k_indices = np.sort(total_top_k_indices)
        # remove duplicates
        total_top_k_indices = np.unique(total_top_k_indices)
        # this might be wrong, it seems like shap returns indices outside of the token range
        # so I'm not sure if shap is using the same tokenization function as ours.
        total_top_k_indices = total_top_k_indices[total_top_k_indices < 2048]
        
        # create a array of the same shape of total_top_k_indices and fill with value i
        index_array = np.full(total_top_k_indices.shape, i)
        
        if i == 0:
            indices_array = [index_array.tolist(), total_top_k_indices.tolist()]
        else:
            # append index array to indices array[0]
            indices_array[0] = indices_array[0] + index_array.tolist()
            # append total_top_k_indices to indices array[1]
            indices_array[1] = indices_array[1] + total_top_k_indices.tolist()
    
    return np.array(indices_array).astype(int)

In [32]:
def predictor_model_token(texts, model, tokenizer):
    # print(len(texts))
    tk = tokenizer(texts, return_tensors="pt",truncation=True, padding=True, max_length=MAX_POSITION_EMBEDDINGS).to(device)
    print(type(tokenizer(texts, return_tensors="pt",truncation=True, padding=True, max_length=MAX_POSITION_EMBEDDINGS)))
    print("token_att: ", dir(tk))
    outputs = model(**tk)
    tensor_logits = outputs[0]
    probas = tensor_logits.sigmoid().detach().cpu().numpy()
    return probas

def predictor_model_no_token(texts, model, tokenizer):
    # print(len(texts))
    # tk = tokenizer(texts, return_tensors="pt",truncation=True, padding=True, max_length=MAX_POSITION_EMBEDDINGS).to(device)
    # tokenization is removed but still need to set texts to device
    # i'm not sure why this is a list and don't have time to debug
    print("Texts_type:", type(texts))
    print("Texts_dir:",  dir(texts))
    texts.to(device)
    outputs = model(**texts)
    tensor_logits = outputs[0]
    probas = tensor_logits.sigmoid().detach().cpu().numpy()
    return probas

In [ ]:
import faithfulness
# this reimports the library for easy testing in the notebook
import importlib
importlib.reload(faithfulness)

MAX_LEN=2048
   
# tokenize the test dataset
test_data = untokenized_dataset["train"][:5]["text"]

inputs = tokenizer(test_data, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')
print(inputs['input_ids'][0][2:30])
print("input type: ", type(inputs))

indices_array = get_index_arrays_shap(test_data, predictor_model_token, model, tokenizer)
print(indices_array)

inputs_rationale_removed = faithfulness.remove_rationale_words(inputs, indices_array, join=False, tokenized=True)
inputs_other_removed = faithfulness.remove_other_words(inputs, indices_array, join=False, tokenized=True)

print("rational removed type: ", type(inputs_rationale_removed))
print("other removed type: ", type(inputs_other_removed))

ind, faith = faithfulness.calculate_faithfulness(inputs, [inputs_rationale_removed], [inputs_other_removed ], model, tokenizer, predictor_model_no_token, tokenized=True)